# Secret Sharing

Secret sharing is a technique widely used in secure multiparty computation. It relies on sharing the responsability of a certain information by simply splitting it in several shares. As always we explain by example.

1. In the first part we will work on additive secret sharing
2. A more complex technique, polynomial reconstruction and Shamir secret sharing.


## Additive Secret Sharing

As said previously, secret sharing consists (at least at this stage) in splitting the information into shares. First let's choose a random integer all the parties agree on:

In [1]:
from crypt import RandomPrime
from random import seed, randrange

seed(1)

bits = 32
p = RandomPrime(bits, 100)

# Although in this example we work with a random number it is not strictly necessary
print(f"We've chosen a random prime p={p}")

We've chosen a random prime p=2272212871


We choose the number of parties $N$ that will participate in the calculation and the secret we want to split. Then we generate N-1 random numbers in between 0 and $p$ and finally substract the secret to generate the Nth share, this way when we sum the shares they will add up to the secret.

In [2]:
N = 5
secret = 100

def AdditiveShares(secret, N, p):
    # generate N-1 random shares
    shares = [randrange(p) for _ in range(N-1)]
    shares.append((secret-sum(shares))%p)
    return shares

def ReconstructAdditiveShares(shares, p):
    return (sum(shares)%p)

shares = AdditiveShares(secret, N, p)
reconstruction = ReconstructAdditiveShares(shares, p)
print(f"Splitting to shares:\n\nsecret={secret}\nsplits={shares}\nsum_shares={reconstruction}")

Splitting to shares:

secret=100
splits=[298566280, 719279514, 685586411, 1096144879, 1744848758]
sum_shares=100


What we've done here is splitting the secret into N users so that when they all share their shares they can reconstruct the secret, no $N-1$ parties can reconstruct!. Beautiful right?

Now, how much information can $N-1$ parties gain from the original secret? Here's the best, there's no information. This is true due to the randomicity of this process, since $N-1$ are chosen completely at random from a uniform distribution they contain no information and by construction the Nth number also. This concept is exactly the same as the one time pad encoding we have seen previously.



## Defining sum on shares

We can define addition and substraction on two vectors of shares. 

In [3]:
def AdditiveSharesAdd(x, y, p):
    return [(i+j)%p for i, j in zip(x,y)]

def AdditiveSharesSubstract(x, y, p):
    return [(i-j)%p for i, j in zip(x,y)]

It is possible to outsource a summation or a substraction by sharing the values of two integers in the following way. Supplose that Alice has the value $x$ and Bob the value $y$, both want to calculate $x+y$ without the other knowing their value (actually, if they know $x+y$ and one value they can know the other value). First what they do is split into $N$ shares:

In [4]:
x = randrange(0, p//2-1)
y = randrange(0, p//2-1)

print(f"I want to add secretly x and y:\nx={x}\ny={y}\n")
print(f"First task is to generate random shares for x and y and send them to the {N} outsourced parties:")
x_shares = AdditiveShares(x, N, p)
y_shares = AdditiveShares(y, N, p)

x_rec = ReconstructAdditiveShares(x_shares, p)
y_rec = ReconstructAdditiveShares(y_shares, p)

print(f"x_shares = {x_shares} which reconstructed is {x_rec}")
print(f"y_shares = {y_shares} which reconstructed is {y_rec}\n")

I want to add secretly x and y:
x=1132540770
y=361088521

First task is to generate random shares for x and y and send them to the 5 outsourced parties:
x_shares = [1172158589, 1264836847, 1952928634, 1382987043, 2176268270] which reconstructed is 1132540770
y_shares = [2132424030, 2034831019, 490445119, 101509731, 146304364] which reconstructed is 361088521



Now Alice has to send each share of $x$ and Bob its shares of $y$ to a different party

In [5]:
for i, (xs, ys) in enumerate(zip(x_shares, y_shares)):
    print(f"Party {i} gets the values x_{i}={xs} and y_{i}={ys}")

Party 0 gets the values x_0=1172158589 and y_0=2132424030
Party 1 gets the values x_1=1264836847 and y_1=2034831019
Party 2 gets the values x_2=1952928634 and y_2=490445119
Party 3 gets the values x_3=1382987043 and y_3=101509731
Party 4 gets the values x_4=2176268270 and y_4=146304364


Each party calculates their sum of their shares and send it back to Alice

In [6]:
x_y_sum_shares = AdditiveSharesAdd(x_shares, y_shares, p)
print(f"The final sum of shares is: \nx_y_sum={x_y_sum_shares}")

The final sum of shares is: 
x_y_sum=[1032369748, 1027454995, 171160882, 1484496774, 50359763]


Now is time to reconstruct the shares and find the secret

In [7]:
x_y_sum = ReconstructAdditiveShares(x_y_sum_shares, p)

assert(x+y==x_y_sum)

print(f"The reconstruction of the secret is {x_y_sum}, the sum of x={x} and y={y}")

The reconstruction of the secret is 1493629291, the sum of x=1132540770 and y=361088521


Done!. We've split two secrets, sent them to different parties that sum their shares and finally we get each contribution in a vector. We just need to add this vector to reconstruct as we did.


### Problems with overflow

You may notice that we've generated two random numbers in the range of $0$ to $p//2-1$, we do this to avoid overflow, this is, if we add two numbers and the sum is larger than $p$ then when we apply the modulo operation we are back to square one and the sum has no meaning. Let me show an example

In [8]:
from crypt import isPrime

# Choose a small prime
q = 17 
assert isPrime(q), f"{q} is not prime, please choose a small prime"
print(f"Prime number selected: q={q}")

# Choose two numbers smaller than p so that when are summed are larger than q
x, y = 10, 8

print(f"Chosen x, y = {x}, {y}")
print(f"Modulo sum of x, y = (x+y)mod q = {(x+y)%q}")

if x+y!=(x+y)%q:
    print("Overflow!")

Prime number selected: q=17
Chosen x, y = 10, 8
Modulo sum of x, y = (x+y)mod q = 1
Overflow!


Let's do the same sum but using the shares

In [9]:
x_shares = AdditiveShares(x, N, q)
y_shares = AdditiveShares(y, N, q)

x_rec = ReconstructAdditiveShares(x_shares, q)
y_rec = ReconstructAdditiveShares(y_shares, q)

print(f"x_shares = {x_shares} which reconstructed is {x_rec}")
print(f"y_shares = {y_shares} which reconstructed is {y_rec}\n")

x_shares = [9, 12, 10, 13, 0] which reconstructed is 10
y_shares = [6, 8, 3, 8, 0] which reconstructed is 8



In [10]:
for i, (xs, ys) in enumerate(zip(x_shares, y_shares)):
    print(f"Party {i} gets the values x_{i}={xs} and y_{i}={ys}")

Party 0 gets the values x_0=9 and y_0=6
Party 1 gets the values x_1=12 and y_1=8
Party 2 gets the values x_2=10 and y_2=3
Party 3 gets the values x_3=13 and y_3=8
Party 4 gets the values x_4=0 and y_4=0


In [11]:
x_y_sum_shares = AdditiveSharesAdd(x_shares, y_shares, q)
x_y_sum = ReconstructAdditiveShares(x_y_sum_shares, q)

print(f"The reconstruction of the secret is {x_y_sum}, the sum of x={x} and y={y}")
if x+1!=x_y_sum:
    print(f"Overflow!, i.e. bad reconstruction.")

The reconstruction of the secret is 1, the sum of x=10 and y=8
Overflow!, i.e. bad reconstruction.


## Defining multiplication on shares

It is possible to define a multiplication in this additive scheme, we will use N=3 parties for simplicity.

The initial step is the same as before, Alice splits $x$ into shares while Bob does the same with $y$. Then they both send the values to each party, in this case:

Party 0: $x_1$, $x_2$, $y_1$, $y_2$

Party 1: $x_2$, $x_0$, $y_2$, $y_0$

Party 2: $x_0$, $x_1$, $y_0$, $y_1$

Nottice that, no party can fully reconstruct x or y, however here if two parties agree they can reconstruct a variable. For instance, if Party 0 and Party 1 are both malitious they can fully reconstruct x and y.


Recall that x = [$x_0$, $x_1$, $x_2$] and y = [$y_0$, $y_1$, $y_2$]
and 

$$x * y = x_1*y_1+x_1*y_2+x_2*y_1+$$
$$x_2*y_0+x_2*y_2+x_0*y_2+$$
$$+x_0*y_0+x_0*y_1+x_1*y_0$$

Party 0 can compute line 0 (containing indexes 1 and 2), Party 1 can compute line 1 (contain indexes 0 and 2) and party 2 calculates line 2 (again contains indexes 0 and 1). Recall the permutation nature of this calculation. We define the following quantities:


$$z_0 = x_1*y_1+x_1*y_2+x_2*y_1$$
$$z_1 = x_2*y_0+x_2*y_2+x_0*y_2$$
$$z_2 = x_0*y_0+x_0*y_1+x_1*y_0$$

whose sum is $x*y$

$$x*y = z = z_0 + z_1 + z_2$$

In [74]:
N = 3

x = 10
y = 17

# Definition of the crossed terms
def indexes(i):
    if i==0:
        return [1, 2]
    if i==1:
        return [2, 0]
    if i==2:
        return [0, 1]
    return None

def SharesToi(v, party):
    i, j = indexes(party)
    return [v[i], v[j]]

def LocalMult(shares, p):   
    return AdditiveShares(shares[0]*shares[2]+shares[0]*shares[3]+shares[1]*shares[2], N, p)

In [75]:
x_shares = AdditiveShares(x, N, p)
y_shares = AdditiveShares(x, N, p)

party_shares_send = []

for party in range(N):
    sharex0, sharex1 = SharesToi(x_shares, party)
    sharey0, sharey1 = SharesToi(y_shares, party)
    
    party_shares_send.append([sharex0, sharex1, sharey0, sharey1])

In [76]:
party_shares_send

[[312214448, 2161357666, 1749384769, 1693119088],
 [2161357666, 2070853638, 1693119088, 1101921895],
 [2070853638, 312214448, 1101921895, 1749384769]]

In [77]:
z_shared = []

for elem in party_shares_send:
    z_shared.append(LocalMult(elem, p))

In [78]:
z_shared

[[864850215, 35561698, 1990326120],
 [1635165130, 2208832078, 1635279890],
 [2091974696, 328191011, 570883617]]

In [80]:
w = [ sum(row) % p for row in zip(*z_shared) ]
print(w)

[47564299, 300371916, 1924276756]


In [65]:
def mul(x, y, p):
    # local computation
    z0 = (x[0]*y[0] + x[0]*y[1] + x[1]*y[0]) % p
    z1 = (x[1]*y[1] + x[1]*y[2] + x[2]*y[1]) % p
    z2 = (x[2]*y[2] + x[2]*y[0] + x[0]*y[2]) % p
    # reshare and distribute; this requires communication
    Z = [ AdditiveShares(z0, 3, p), AdditiveShares(z1, 3, p), AdditiveShares(z2, 3, p) ]
    w = [ sum(row) % p for row in zip(*Z) ]
    # bring precision back down from double to single
    return w

kk = mul(AdditiveShares(4, 3, p), AdditiveShares(5, 3, p), p)
sum(kk)%p

20